# Phishing Dataset: Parsing, Cleaning, and Wrangling

## Importing libraries

In [1]:
import pandas as pd
from pathlib import Path
from functools import reduce

## Loading the files

In [2]:
data_path = Path().parent

# PhiUSIIL url dataset
url_dataset = list(data_path.rglob("*url_dataset.csv"))[0]

# Multiple files
phishing_email_datasets = list(data_path.rglob("*dataset/*.csv"))

In [3]:
url_df = pd.read_csv(url_dataset)
url_df.head()

,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,100.0,1.000000,0.522907,0.061933,...,0,0,1,34,20,28,119,0,124,1
1,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,100.0,0.666667,0.032650,0.050207,...,0,0,1,50,9,8,39,0,217,1
2,https://www.voicefmradio.co.uk,29,www.voicefmradio.co.uk,22,0,uk,100.0,0.866667,0.028555,0.064129,...,0,0,1,10,2,7,42,2,5,1
3,https://www.sfnmjournal.com,26,www.sfnmjournal.com,19,0,com,100.0,1.000000,0.522907,0.057606,...,1,1,1,3,27,15,22,1,31,1
4,https://www.rewildingargentina.org,33,www.rewildingargentina.org,26,0,org,100.0,1.000000,0.079963,0.059441,...,1,0,1,244,15,34,72,1,85,1


In [4]:
# Create iterator
dfs = (pd.read_csv(csv) for csv in phishing_email_datasets)

# Concatenate into single df since all follow same format
email_df = reduce(lambda df1, df2: pd.concat([df1, df2]), dfs)
email_df = email_df.reset_index(drop=True)
email_df = email_df.drop(columns=["body", "text_combined"], axis=1)

email_df.head()

,sender,receiver,date,subject,urls,label
0,MR. JAMES NGOLA. <james_ngola2002@maktoob.com>,webmaster@aclweb.org,"Thu, 31 Oct 2002 02:38:20 +0000",URGENT BUSINESS ASSISTANCE AND PARTNERSHIP,0.0,1
1,Mr. Ben Suleman <bensul2004nng@spinfinder.com>,R@M,"Thu, 31 Oct 2002 05:10:00 -0000",URGENT ASSISTANCE /RELATIONSHIP (P),0.0,1
2,PRINCE OBONG ELEME <obong_715@epatra.com>,webmaster@aclweb.org,"Thu, 31 Oct 2002 22:17:55 +0100",GOOD DAY TO YOU,0.0,1
3,PRINCE OBONG ELEME <obong_715@epatra.com>,webmaster@aclweb.org,"Thu, 31 Oct 2002 22:44:20 -0000",GOOD DAY TO YOU,0.0,1
4,Maryam Abacha <m_abacha03@www.com>,R@M,"Fri, 01 Nov 2002 01:45:04 +0100",I Need Your Assistance.,0.0,1


## Parsing and wrangling URLs

In [5]:
# Select only important parts
url_df_clean = url_df[["URL", "Domain", "label"]].copy()

# What's before ://
url_df_clean["scheme"] = url_df.URL.str.extract(r"(.+)(?=:[/]{2})")

# After https://www.
url_df_clean["registrable_domain"] = url_df.URL.str.extract(r"(?<=:[/]{2})(?:\w+\.)?(.+)(?=/?)")

# From PhiUSIIL
url_df_clean["source"] = "https://archive.ics.uci.edu/dataset/967/phiusiil+phishing+url+dataset"

# 1 = legitimate, 0 = illegitimate
url_df_clean.loc[url_df_clean["label"] == 0, "label"] = 3 # Mark as high risk
url_df_clean.loc[~url_df_clean["label"].isin([1, 3]), "label"] = 0 # Mark as unknown risk; retain 1 (legitimate) as low risk

# Rename cols same as schema and rearrange
url_df_clean = url_df_clean.rename(columns={"URL": "full_url", "Domain": "host", "label": "risk_level"})
url_df_clean = url_df_clean[["scheme", "host", "registrable_domain", "full_url", "risk_level", "source"]]
url_df_clean.head()

,scheme,host,registrable_domain,full_url,risk_level,source
0,https,www.southbankmosaics.com,southbankmosaics.com,https://www.southbankmosaics.com,1,https://archive.ics.uci.edu/dataset/967/phiusi...
1,https,www.uni-mainz.de,uni-mainz.de,https://www.uni-mainz.de,1,https://archive.ics.uci.edu/dataset/967/phiusi...
2,https,www.voicefmradio.co.uk,voicefmradio.co.uk,https://www.voicefmradio.co.uk,1,https://archive.ics.uci.edu/dataset/967/phiusi...
3,https,www.sfnmjournal.com,sfnmjournal.com,https://www.sfnmjournal.com,1,https://archive.ics.uci.edu/dataset/967/phiusi...
4,https,www.rewildingargentina.org,rewildingargentina.org,https://www.rewildingargentina.org,1,https://archive.ics.uci.edu/dataset/967/phiusi...


## Parsing and wrangling emails

In [6]:
# Follow same format as the sql schema
email_df_clean = pd.DataFrame()
email_df_clean["address"] = email_df.sender.str.lower().str.extract(r"<(.+@.+)>")
email_df_clean.insert(0, "local_part", email_df_clean.address.str.extract(r"(.+)(?=@.+)"))
email_df_clean.insert(1, "domain", email_df_clean.address.str.extract(r"(?<=@)(.+)"))

# Legitimate = 1, Phishing = 0
email_df_clean["risk_level"] = email_df["label"].copy()

# Convert phishing email to high risk; retain legitimate as low risk
email_df_clean.loc[email_df_clean["risk_level"] == 0, "risk_level"] = 3

# Add source and notes (sender name)
email_df_clean["source"] = "https://www.kaggle.com/datasets/naserabdullahalam/phishing-email-dataset/data"
email_df_clean["notes"] = "<sender>" + email_df.sender.str.lower().str.extract(r"(.+?)\.?\s?(?=<.+>)") + "</sender>"

email_df_clean.head()

,local_part,domain,address,risk_level,source,notes
0,james_ngola2002,maktoob.com,james_ngola2002@maktoob.com,1,https://www.kaggle.com/datasets/naserabdullaha...,<sender>mr. james ngola</sender>
1,bensul2004nng,spinfinder.com,bensul2004nng@spinfinder.com,1,https://www.kaggle.com/datasets/naserabdullaha...,<sender>mr. ben suleman</sender>
2,obong_715,epatra.com,obong_715@epatra.com,1,https://www.kaggle.com/datasets/naserabdullaha...,<sender>prince obong eleme</sender>
3,obong_715,epatra.com,obong_715@epatra.com,1,https://www.kaggle.com/datasets/naserabdullaha...,<sender>prince obong eleme</sender>
4,m_abacha03,www.com,m_abacha03@www.com,1,https://www.kaggle.com/datasets/naserabdullaha...,<sender>maryam abacha</sender>


## Exporting the clean data

In [7]:
clean_fp = data_path / "clean"
if not clean_fp.exists():
    clean_fp.mkdir(parents=True)

email_df_clean.to_parquet(clean_fp / "email_phishing_clean.parquet")
url_df_clean.to_parquet(clean_fp / "url_phishing_clean.parquet")

In [ ]:
# Test if files are not corrupted
pd.read_parquet(clean_fp / "email_phishing_clean.parquet").head()

,local_part,domain,address,risk_level,source,notes
0,james_ngola2002,maktoob.com,james_ngola2002@maktoob.com,1,https://www.kaggle.com/datasets/naserabdullaha...,<sender>mr. james ngola</sender>
1,bensul2004nng,spinfinder.com,bensul2004nng@spinfinder.com,1,https://www.kaggle.com/datasets/naserabdullaha...,<sender>mr. ben suleman</sender>
2,obong_715,epatra.com,obong_715@epatra.com,1,https://www.kaggle.com/datasets/naserabdullaha...,<sender>prince obong eleme</sender>
3,obong_715,epatra.com,obong_715@epatra.com,1,https://www.kaggle.com/datasets/naserabdullaha...,<sender>prince obong eleme</sender>
4,m_abacha03,www.com,m_abacha03@www.com,1,https://www.kaggle.com/datasets/naserabdullaha...,<sender>maryam abacha</sender>


In [8]:
pd.read_parquet(clean_fp / "url_phishing_clean.parquet").head()

,scheme,host,registrable_domain,full_url,risk_level,source
0,https,www.southbankmosaics.com,southbankmosaics.com,https://www.southbankmosaics.com,1,https://archive.ics.uci.edu/dataset/967/phiusi...
1,https,www.uni-mainz.de,uni-mainz.de,https://www.uni-mainz.de,1,https://archive.ics.uci.edu/dataset/967/phiusi...
2,https,www.voicefmradio.co.uk,voicefmradio.co.uk,https://www.voicefmradio.co.uk,1,https://archive.ics.uci.edu/dataset/967/phiusi...
3,https,www.sfnmjournal.com,sfnmjournal.com,https://www.sfnmjournal.com,1,https://archive.ics.uci.edu/dataset/967/phiusi...
4,https,www.rewildingargentina.org,rewildingargentina.org,https://www.rewildingargentina.org,1,https://archive.ics.uci.edu/dataset/967/phiusi...
